In [1]:
import os
from os import listdir
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
import re
import pathlib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyheif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 18.3 MB/s eta 0:00:00


In [4]:
# Define your dataset directory
Agriculture_crop = '/content/drive/MyDrive/crop_images'

# Define types
types = [
    "jute", "maize", "rice", "sugarcane", "wheat"
]

# Initialize lists to hold images and their labels
CrownPreparationimages = []
CrownPreparationlabels = []



target_size = (224, 224)  # Desired size to resize images

for i, PreparationType in enumerate(types, start=1):
    CrownPreparationFolder = os.path.join(Agriculture_crop, str(i))
    for root, dirs, files in os.walk(CrownPreparationFolder):
        for file_name in files:
            img_path = os.path.join(root, file_name)
            _, file_extension = os.path.splitext(img_path)
            if file_extension.lower() == '.jpeg':
            #for add photo output folder
            #if file_extension.lower() == '.png':
                img = Image.open(img_path)
            elif file_extension.lower() == '.heic':
                try:
                    heif_file = pyheif.read(img_path)
                    img = Image.frombytes(
                        heif_file.mode,
                        heif_file.size,
                        heif_file.data,
                        "raw",
                        heif_file.mode,
                        heif_file.stride,
                    )
                except Exception as e:
                    print(f"Error opening {img_path}: {e}")
                    continue
            else:
                print(f"Unsupported file format: {img_path}")
                continue

            # Resize image to target size
            img = img.resize(target_size)

            # Append image and label
            CrownPreparationimages.append(np.array(img))
            CrownPreparationlabels.append(PreparationType)

CrownPreparationimages = np.array(CrownPreparationimages)
CrownPreparationlabels = np.array(CrownPreparationlabels)

In [5]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

print("Number of images loaded:", len(CrownPreparationimages))
print("Number of labels:", len(CrownPreparationlabels))


Number of images loaded: 804
Number of labels: 804


In [ ]:
# Define the ResNet model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense


# Split the dataset into training and testing
train_images, test_images, train_labels, test_labels = train_test_split(
    CrownPreparationimages, CrownPreparationlabels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
label_binarizer = LabelBinarizer()
train_labels = label_binarizer.fit_transform(train_labels)
test_labels = label_binarizer.transform(test_labels)

img_width, img_height = 224, 224
batch_size = 16

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

def build_resnet(input_shape, num_classes):
    resnet = tf.keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )

    for layer in resnet.layers:
        layer.trainable = False

    model = models.Sequential([
        resnet,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Build the model
model = build_resnet((img_width, img_height, 3), num_classes=len(types))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=100,
                    validation_data=(test_images, test_labels),
                    callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/100
21/21 [==============================] - 207s 9s/step - loss: 1.5428 - accuracy: 0.3857 - val_loss: 0.9824 - val_accuracy: 0.6149
Epoch 2/100
21/21 [==============================] - 171s 8s/step - loss: 0.6580 - accuracy: 0.8025 - val_loss: 0.5998 - val_accuracy: 0.8571
Epoch 3/100
21/21 [==============================] - 165s 8s/step - loss: 0.4115 - accuracy: 0.8958 - val_loss: 0.4764 - val_accuracy: 0.9006
Epoch 4/100
21/21 [==============================] - 174s 8s/step - loss: 0.2844 - accuracy: 0.9549 - val_loss: 0.3437 - val_accuracy: 0.9379
Epoch 5/100
21/21 [==============================] - 194s 9s/step - loss: 0.2075 - accuracy: 0.9813 - val_loss: 0.2813 - val_accuracy: 0.9503
Epoch 6/100
21/21 [==============================] - 173s 8s/step - loss: 0.1719 - accuracy: 0.9876 - val_loss: 0.2450 - val_accuracy: 0.9441
Epoch 7/100
21/21 [==============================] - 173s 8s/step - loss: 0.1405 - accuracy: 0.9922 - val_loss: 0.2034 - val_accuracy: 0.9752
Epoch 